In [1]:
# 원본 json 파일에서 필요한 정보들 모아서 df로 만든 후 저장

import pandas as pd
import numpy as np
from google.colab import drive 
import os
import json
import tqdm

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
temp_dir=os.listdir('/content/drive/MyDrive/객체인식/data_new/train/label/001')[:30]
temp_dir[0]

'0000005_as-0036229.json'

In [ ]:
image_folders=os.listdir('/content/drive/MyDrive/객체인식/data_new/train/image')


In [86]:
#temp

for image_folder in tqdm.tqdm(image_folders[:1]):
    dirs=os.listdir('/content/drive/MyDrive/객체인식/data_new/train/label/'+image_folder)
    for current_dir in dirs:
        my_path='/content/drive/MyDrive/객체인식/data_new/train/label/'+image_folder+'/'+current_dir

        with open(f'{my_path}', 'r') as f:         # save_path : bbox.txt 파일 저장할 경로
            json_data = json.load(f)


        temp_bbox=[]
        temp_segmentation=[]
        temp_area=[]
        scratched, separated, breakage, crushed=0,0,0,0

        #손상 심각도도 필요할까?
            
        #for문을 돌면서 bbox, segmentation, area 값을 모으자
        for i in range(len(json_data['annotations'])):
            temp_bbox.append(json_data['annotations'][i]['bbox'])
            temp_segmentation.append(sum(sum(json_data['annotations'][i]['segmentation'],[]),[]))
            temp_area.append(json_data['annotations'][i]['area'])
                
            #'Scratched'Separated Breakage Crushed
            if json_data['annotations'][i]['damage']=="Scratched":
                scratched=1
            elif json_data['annotations'][i]['damage']=="Separated":
                separated=1
            elif json_data['annotations'][i]['damage']=="Breakage":
                breakage=1
            elif json_data['annotations'][i]['damage']=="Crushed":
                crushed=1
            else:
                print("outlier: json_data['annotations'][i]['damage']",json_data['annotations'][i]['damage'])
   

json_data['annotations'][0]['part']

In [31]:
df=pd.DataFrame(columns={'folder','filename','width','height','segmentation', 'repair',
                         'bbox','area','scratched', 'separated', 'breakage', 'crushed'})

for image_folder in tqdm.tqdm(image_folders):
    dirs=os.listdir('/content/drive/MyDrive/객체인식/data_new/train/label/'+image_folder)
    for current_dir in dirs:
        my_path='/content/drive/MyDrive/객체인식/data_new/train/label/'+image_folder+'/'+current_dir

        with open(f'{my_path}', 'r') as f:         # save_path : bbox.txt 파일 저장할 경로
            json_data = json.load(f)


        temp_bbox=[]
        temp_segmentation=[]
        temp_area=[]
        scratched, separated, breakage, crushed=0,0,0,0

        #손상 심각도도 필요할까?
            
        #for문을 돌면서 bbox, segmentation, area 값을 모으자
        for i in range(len(json_data['annotations'])):
            temp_bbox.append(json_data['annotations'][i]['bbox'])
            temp_segmentation.append(sum(sum(json_data['annotations'][i]['segmentation'],[]),[]))
            temp_area.append(json_data['annotations'][i]['area'])
                
            #'Scratched'Separated Breakage Crushed
            if json_data['annotations'][i]['damage']=="Scratched":
                scratched=1
            elif json_data['annotations'][i]['damage']=="Separated":
                separated=1
            elif json_data['annotations'][i]['damage']=="Breakage":
                breakage=1
            elif json_data['annotations'][i]['damage']=="Crushed":
                crushed=1
            else:
                print("outlier: json_data['annotations'][i]['damage']",json_data['annotations'][i]['damage'])
            
        #파손부위 list 만들기
        #print(json_data['annotations'][0]['repair'])
        if json_data['annotations'][0]['repair']:
            temp_repair=[x.lower().split(':')[0] for x in json_data['annotations'][0]['repair']]
            temp_repair=[x.replace('rear', '').replace('front','').replace('(l)','').replace('(r)','').
                         replace('a pillar', 'pillar').replace('b pillar', 'pillar').replace('c pillar', 'pillar').strip() for x in temp_repair]
            #겹칠 때 하나만 
            ###########
            temp_repair=list(set(temp_repair))
        else:
            temp_repair = []

        #합치기
        new_row = {
        'folder': image_folder,
        'filename':json_data['images']['file_name'].split('.')[0],
        'width':json_data['images']['width'], 
        'height':json_data['images']['height'],
        'segmentation':temp_segmentation,
        'repair':temp_repair,
        'bbox':temp_bbox,
        'area':temp_area,
        'scratched':scratched, 
        'separated':separated, 
        'breakage':breakage, 
        'crushed':crushed
        }
            
        df = pd.concat([df, pd.DataFrame([new_row], columns=df.columns)], ignore_index=True)

    

 25%|██▍       | 105/423 [1:02:11<3:08:21, 35.54s/it]


FileNotFoundError: ignored

In [37]:
#순서가 엉망이라 정리해줌
df=df[['folder','filename','width','height','segmentation','area','bbox','repair','scratched', 'separated', 'breakage', 'crushed']]
df

,folder,filename,width,height,segmentation,area,bbox,repair,scratched,separated,breakage,crushed
0,090,0113498_as-2936585,800,600,"[[[0, 183], [2, 1], [241, 3], [323, 3], [498, ...","[129748.5, 99659.0, 5637.5, 49906.0]","[[0, 1, 571, 335], [0, 173, 522, 358], [519, 3...","[bumper, pillar, door]",1,1,1,1
1,090,0113411_sc-197864,800,600,"[[[362, 336], [365, 347], [388, 339], [390, 33...","[258.5, 634.0, 157.5, 75.0, 917.5, 1551.5]","[[362, 330, 28, 17], [284, 433, 29, 43], [367,...",[bumper],1,1,1,0
2,090,0113330_sc-123247,800,600,"[[[321, 226], [372, 249], [356, 258], [326, 24...",[718.5],"[[321, 226, 51, 32]]",[bumper],1,0,0,0
3,090,0113501_sc-210276,800,600,"[[[32, 2], [46, 39], [73, 66], [111, 76], [122...","[8415.5, 337.5, 3112.0, 7201.0, 3568.0, 237.5,...","[[0, 0, 154, 120], [266, 134, 29, 30], [73, 28...","[bumper, fender, door]",1,0,0,0
4,090,0113595_as-3383916,800,600,"[[[4, 471], [0, 433], [255, 331], [431, 316], ...","[21290.5, 4113.0]","[[0, 316, 431, 155], [610, 257, 190, 126]]","[door, wheel, bumper, fender, side mirror]",1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
104995,105,0133546_sc-1004029,800,600,"[[[378, 2], [627, 2], [640, 20], [681, 32], [6...","[18722.0, 10248.0, 12574.0]","[[378, 2, 305, 80], [85, 129, 394, 214], [282,...",[door],1,1,0,1
104996,105,0133547_sc-1004029,800,600,"[[[409, 271], [462, 259], [447, 314], [383, 34...","[4175.5, 479.0, 651.0]","[[369, 259, 93, 83], [470, 227, 23, 36], [369,...",[door],1,1,0,1
104997,105,0133549_sc-137784,800,600,"[[[97, 366], [284, 280], [357, 249], [578, 200...","[28376.0, 804.0]","[[97, 180, 552, 202], [575, 541, 36, 53]]",[bumper],1,0,0,0
104998,105,0133550_sc-137784,800,600,"[[[70, 417], [330, 386], [401, 370], [450, 332...",[9624.5],"[[69, 332, 388, 110]]",[bumper],1,0,0,0


In [81]:
temp_repair_df=df['repair'].value_counts().to_frame()
temp_repair_df.reset_index(inplace=True)
temp_repair_df['index']=temp_repair_df['index'].to_list()
temp_repair_df[temp_repair_df['index'].apply(lambda x: len(x) < 2)]

,index,repair
0,[bumper],36155
1,[],8564
5,[door],3866
19,[fender],791
35,[wheel],298
41,[rocker panel],255
75,[roof],89
77,[side mirror],86
90,[bonnet],57
109,[trunk lid],38


In [ ]:
# 각각 몇개 있는지

df['repair']
for i in my_list:
    



In [34]:
print(df['scratched'].value_counts())
print(df['separated'].value_counts())
print(df['breakage'].value_counts())
print(df['crushed'].value_counts())


1    93571
0    11429
Name: scratched, dtype: int64
0    68468
1    36532
Name: separated, dtype: int64
0    82129
1    22871
Name: breakage, dtype: int64
0    77078
1    27922
Name: crushed, dtype: int64


In [ ]:
#확인해 보아야 할 것
#사진에 없는 부분이 repair 되지는 않았는지

#bbox : defect
#bbox별로 어디 파손인지 안나옴
#bbox 개수와 repair 수와 손상 수가 다름
#->repair을 라벨로 사용하지 못할듯

In [83]:
#save data
#
df.to_pickle('/content/drive/MyDrive/객체인식/outputs/json_to_df.pkl')
#df.to_pickle('json_to_df.pkl')